In [1]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
import keras.backend as K
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

In [2]:
# Store the input data in a dataframe
java_data = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/data_Java.csv")
c_data = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/data_C.csv")
php_data = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/data_PHP.csv")
python_data = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/data_Python.csv")
ruby_data = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/data_Ruby.csv")


In [3]:
# Store the input labels in a dataframe
java_labels = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/label_Java.csv")
c_labels = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/label_C.csv")
php_labels = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/label_PHP.csv")
python_labels = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/label_Python.csv")
ruby_labels = pd.read_csv("C:/Users/victoriawu/Hackathon2021/conflict-prediction/replication_package/prediction_data/label_Ruby.csv")


In [21]:
#Append all data
df = pd.DataFrame()
df = df.append(java_data)
df = df.append(c_data)
df = df.append(php_data)
df = df.append(python_data)
df = df.append(ruby_data)

In [22]:
# create an abs_scaler object
abs_scaler = MaxAbsScaler()

# calculate the maximum absolute value for scaling the data using the fit method
abs_scaler.fit(df)

# transform the data using the parameters calculated by the fit method (the maximum absolute values)
scaled_data = abs_scaler.transform(df)

# store the results in a data frame
df_scaled = pd.DataFrame(scaled_data, columns=df.columns)

# visualize the data frame
df_scaled

,parallel_changed_file_num,commit_num,file_added,file_removed,file_renamed,file_modified,file_copied,line_added,line_removed,developer_num,...,fix_frequency,improve_frequency,refactor_frequency,remove_frequency,update_frequency,use_frequency,messages_min,messages_max,messages_mean,messages_median
0,0.000000,0.000023,0.000000,0.000021,0.0,0.000000e+00,0.0,0.000000e+00,5.773464e-06,0.000733,...,0.000000,0.000000,0.0,0.041667,0.000000,0.000000,0.042394,0.040670,0.042394,0.042394
1,0.000000,0.000340,-0.000084,0.000000,0.0,-4.811897e-05,0.0,-2.114883e-05,-4.792353e-06,-0.002199,...,0.083333,0.000000,0.0,0.041667,0.000000,0.086957,0.039900,0.318182,0.130840,0.119701
2,0.000000,0.000068,-0.000015,0.000007,0.0,1.804461e-06,0.0,-6.687378e-07,3.018805e-07,0.000000,...,0.000000,0.000000,0.0,0.000000,0.022222,0.000000,0.104738,0.148325,0.123026,0.109726
3,0.000000,0.000045,-0.000053,-0.000079,0.0,-2.345800e-05,0.0,-6.453320e-05,-6.373452e-05,0.000000,...,0.000000,0.000000,0.0,0.000000,0.022222,0.000000,0.042394,0.100478,0.073566,0.073566
4,0.000000,0.000883,-0.000496,-0.000114,0.0,-1.383420e-04,0.0,-2.567117e-04,-7.794177e-05,-0.005865,...,0.083333,0.153846,0.0,0.083333,0.000000,0.043478,0.000000,0.155502,0.114476,0.112219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185214,0.000000,0.000023,0.000000,0.000000,0.0,6.014871e-07,0.0,8.359223e-08,5.660259e-08,0.000733,...,0.166667,0.000000,0.0,0.041667,0.000000,0.000000,0.064838,0.217703,0.118821,0.114713
185215,0.000000,0.000068,0.000038,0.000000,0.0,-5.413384e-06,0.0,2.089806e-06,-4.905558e-07,-0.000733,...,0.000000,0.076923,0.0,0.083333,0.000000,0.086957,0.054863,0.145933,0.108728,0.127182
185216,0.000000,0.000770,-0.000839,-0.000079,0.0,-9.443348e-05,0.0,-4.021344e-04,-2.449005e-05,-0.003666,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.082294,0.078947,0.082294,0.082294
185217,0.000000,0.000023,0.000000,0.000000,0.0,6.014871e-07,0.0,1.114563e-07,0.000000e+00,0.000733,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.109726,0.105263,0.109726,0.109726


In [23]:
#Add all labels into one dataframe
labelsdf = pd.DataFrame()
labelsdf = labelsdf.append(java_labels)
labelsdf = labelsdf.append(c_labels)
labelsdf = labelsdf.append(php_labels)
labelsdf = labelsdf.append(python_labels)
labelsdf = labelsdf.append(ruby_labels)

In [34]:
#Append labels to the training data 
df_scaled.insert(28, 'is_conflict', labelsdf)

df_scaled

,parallel_changed_file_num,commit_num,file_added,file_removed,file_renamed,file_modified,file_copied,line_added,line_removed,developer_num,...,improve_frequency,refactor_frequency,remove_frequency,update_frequency,use_frequency,messages_min,messages_max,messages_mean,messages_median,is_conflict
0,0.000000,0.000023,0.000000,0.000021,0.0,0.000000e+00,0.0,0.000000e+00,5.773464e-06,0.000733,...,0.000000,0.0,0.041667,0.000000,0.000000,0.042394,0.040670,0.042394,0.042394,0
1,0.000000,0.000340,-0.000084,0.000000,0.0,-4.811897e-05,0.0,-2.114883e-05,-4.792353e-06,-0.002199,...,0.000000,0.0,0.041667,0.000000,0.086957,0.039900,0.318182,0.130840,0.119701,0
2,0.000000,0.000068,-0.000015,0.000007,0.0,1.804461e-06,0.0,-6.687378e-07,3.018805e-07,0.000000,...,0.000000,0.0,0.000000,0.022222,0.000000,0.104738,0.148325,0.123026,0.109726,0
3,0.000000,0.000045,-0.000053,-0.000079,0.0,-2.345800e-05,0.0,-6.453320e-05,-6.373452e-05,0.000000,...,0.000000,0.0,0.000000,0.022222,0.000000,0.042394,0.100478,0.073566,0.073566,0
4,0.000000,0.000883,-0.000496,-0.000114,0.0,-1.383420e-04,0.0,-2.567117e-04,-7.794177e-05,-0.005865,...,0.153846,0.0,0.083333,0.000000,0.043478,0.000000,0.155502,0.114476,0.112219,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185214,0.000000,0.000023,0.000000,0.000000,0.0,6.014871e-07,0.0,8.359223e-08,5.660259e-08,0.000733,...,0.000000,0.0,0.041667,0.000000,0.000000,0.064838,0.217703,0.118821,0.114713,0
185215,0.000000,0.000068,0.000038,0.000000,0.0,-5.413384e-06,0.0,2.089806e-06,-4.905558e-07,-0.000733,...,0.076923,0.0,0.083333,0.000000,0.086957,0.054863,0.145933,0.108728,0.127182,0
185216,0.000000,0.000770,-0.000839,-0.000079,0.0,-9.443348e-05,0.0,-4.021344e-04,-2.449005e-05,-0.003666,...,0.000000,0.0,0.000000,0.000000,0.000000,0.082294,0.078947,0.082294,0.082294,0
185217,0.000000,0.000023,0.000000,0.000000,0.0,6.014871e-07,0.0,1.114563e-07,0.000000e+00,0.000733,...,0.000000,0.0,0.000000,0.000000,0.000000,0.109726,0.105263,0.109726,0.109726,0


In [35]:
#Shuffle the data to randomize the languages
df_scaled = df_scaled.sample(frac=1).reset_index(drop=True)

In [36]:
# Split the data into training (80%) and testing (20%)
train, test = train_test_split(df_scaled, test_size=0.2)

# Split data into features and labels (only using the first 29 columns as 30 is the amount and 31 is the label
train_features = np.array(train.values[:,:27])
train_labels = np.array(train.values[:,-1])
test_features = np.array(test.values[:,:27])
test_labels = np.array(test.values[:,-1])

In [37]:
model = Sequential()
model.add(Dense(units=100, kernel_initializer='uniform', input_dim=train_features.shape[1], activation='relu'))
model.add(Dropout(.2))
model.add(Dense(units=100, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(.2))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [38]:
train_history = model.fit(train_features, train_labels, batch_size=50, epochs=200, verbose=1, validation_split=0.2)

Epoch 1/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.2811 - accuracy: 0.9198 - val_loss: 0.2569 - val_accuracy: 0.9239
Epoch 2/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.2575 - accuracy: 0.9197 - val_loss: 0.2365 - val_accuracy: 0.9241
Epoch 3/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.2366 - accuracy: 0.9193 - val_loss: 0.2197 - val_accuracy: 0.9246
Epoch 4/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.2259 - accuracy: 0.9202 - val_loss: 0.2068 - val_accuracy: 0.9258
Epoch 5/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.2193 - accuracy: 0.9207 - val_loss: 0.2039 - val_accuracy: 0.9245
Epoch 6/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.2144 - accuracy: 0.9211 - val_loss: 0.2007 - val_accuracy: 0.9261
Epoch 7/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.2100 - accuracy: 0.9217 - val_loss: 0.2039 - val_ac

2371/2371 [==============================] - 5s 2ms/step - loss: 0.1682 - accuracy: 0.9325 - val_loss: 0.1566 - val_accuracy: 0.9355
Epoch 58/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1685 - accuracy: 0.9330 - val_loss: 0.1557 - val_accuracy: 0.9367
Epoch 59/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1680 - accuracy: 0.9329 - val_loss: 0.1555 - val_accuracy: 0.9366
Epoch 60/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1671 - accuracy: 0.9327 - val_loss: 0.1653 - val_accuracy: 0.9355
Epoch 61/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1673 - accuracy: 0.9330 - val_loss: 0.1562 - val_accuracy: 0.9330
Epoch 62/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1672 - accuracy: 0.9327 - val_loss: 0.1566 - val_accuracy: 0.9350
Epoch 63/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1673 - accuracy: 0.9330 - val_loss: 0.1584 - val_accuracy

2371/2371 [==============================] - 5s 2ms/step - loss: 0.1631 - accuracy: 0.9337 - val_loss: 0.1596 - val_accuracy: 0.9355
Epoch 114/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1631 - accuracy: 0.9338 - val_loss: 0.1542 - val_accuracy: 0.9358
Epoch 115/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1627 - accuracy: 0.9335 - val_loss: 0.1579 - val_accuracy: 0.9352
Epoch 116/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1629 - accuracy: 0.9327 - val_loss: 0.1723 - val_accuracy: 0.9355
Epoch 117/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1650 - accuracy: 0.9329 - val_loss: 0.1562 - val_accuracy: 0.9345
Epoch 118/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1633 - accuracy: 0.9341 - val_loss: 0.1547 - val_accuracy: 0.9358
Epoch 119/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1620 - accuracy: 0.9341 - val_loss: 0.1555 - val_ac

Epoch 169/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1604 - accuracy: 0.9345 - val_loss: 0.1545 - val_accuracy: 0.9365
Epoch 170/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1603 - accuracy: 0.9336 - val_loss: 0.1602 - val_accuracy: 0.9349
Epoch 171/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1603 - accuracy: 0.9341 - val_loss: 0.1554 - val_accuracy: 0.9365
Epoch 172/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1604 - accuracy: 0.9342 - val_loss: 0.1552 - val_accuracy: 0.9360
Epoch 173/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1600 - accuracy: 0.9342 - val_loss: 0.1572 - val_accuracy: 0.9367
Epoch 174/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1604 - accuracy: 0.9343 - val_loss: 0.1604 - val_accuracy: 0.9362
Epoch 175/200
2371/2371 [==============================] - 5s 2ms/step - loss: 0.1595 - accuracy: 0.9345 - val_loss: 0

In [39]:
# Get model accuracy 
scores = model.evaluate(test_features, test_labels)
print('\n')
print('accuracy=',scores[1])

# Get predictions from model
output = model.predict_classes(test_features)

# Show confusion matrix
y_actu = pd.Series(test_labels, name='Actual')
y_pred = pd.Series(np.ndarray.flatten(output), name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)

num_positives =  (np.count_nonzero(y_actu))
num_negatives = y_actu.size - num_positives

TN = df_confusion[0][0]
TP = df_confusion[1][1]
FN = df_confusion[0][1]
FP = df_confusion[1][0]

precision = TP/(TP + FP)
recall = TP/(TP + FN)
specificity = TN/(TN + FP)
f1 = 2*((precision*recall)/(precision+recall))
print("Num Fraud: {}".format(num_positives))

print("Precision: ", precision)
print("Recall/Sensitivity: ", recall)
print("Specificity: ", specificity)
print("F1 Score: ", f1)

1158/1158 [==============================] - 1s 986us/step - loss: 0.1628 - accuracy: 0.9349


accuracy= 0.9348612427711487
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Predicted      0     1
Actual                
0.0        33423   646
1.0         1767  1208
Num Fraud: 2975
Precision:  0.651564185544768
Recall/Sensitivity:  0.40605042016806725
Specificity:  0.9810384807302827
F1 Score:  0.500310623317457


In [40]:
#Cost sensitive
class_weights = class_weight.compute_class_weight("balanced", np.unique(train_labels), train_labels)
class_weights = {i : class_weights[i] for i in range(2)}

C:\Users\victoriawu\Anaconda3\envs\MSFTHackathon2021\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0. 1.], y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [41]:
model2 = Sequential()
model2.add(Dense(units=100, kernel_initializer='uniform', input_dim=train_features.shape[1], activation='tanh'))
model2.add(Dropout(.2))
model2.add(Dense(units=100, kernel_initializer='uniform', activation='tanh'))
model2.add(Dropout(.2))
model2.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model2.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [42]:
train_history2 = model2.fit(train_features, train_labels, batch_size=50, epochs=200, verbose=1, validation_split=0.2, class_weight=class_weights)

Epoch 1/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.6699 - accuracy: 0.6369 - val_loss: 0.6701 - val_accuracy: 0.8166
Epoch 2/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.6168 - accuracy: 0.7938 - val_loss: 0.6547 - val_accuracy: 0.8183
Epoch 3/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.5783 - accuracy: 0.8349 - val_loss: 0.4064 - val_accuracy: 0.9044
Epoch 4/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.5551 - accuracy: 0.8430 - val_loss: 0.5115 - val_accuracy: 0.8741
Epoch 5/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.5385 - accuracy: 0.8445 - val_loss: 0.5918 - val_accuracy: 0.8540
Epoch 6/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.5203 - accuracy: 0.8485 - val_loss: 0.4155 - val_accuracy: 0.8878
Epoch 7/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.5055 - accuracy: 0.8469 - val_loss: 0.6216 - val_ac

2371/2371 [==============================] - 6s 2ms/step - loss: 0.3470 - accuracy: 0.8498 - val_loss: 0.3223 - val_accuracy: 0.8445
Epoch 58/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3466 - accuracy: 0.8498 - val_loss: 0.2949 - val_accuracy: 0.8576
Epoch 59/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3472 - accuracy: 0.8498 - val_loss: 0.4069 - val_accuracy: 0.8282
Epoch 60/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3463 - accuracy: 0.8503 - val_loss: 0.3089 - val_accuracy: 0.8487
Epoch 61/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3464 - accuracy: 0.8478 - val_loss: 0.4426 - val_accuracy: 0.8197
Epoch 62/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3455 - accuracy: 0.8476 - val_loss: 0.3318 - val_accuracy: 0.8348
Epoch 63/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3436 - accuracy: 0.8491 - val_loss: 0.3290 - val_accuracy

2371/2371 [==============================] - 5s 2ms/step - loss: 0.3327 - accuracy: 0.8412 - val_loss: 0.2857 - val_accuracy: 0.8506
Epoch 114/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3312 - accuracy: 0.8428 - val_loss: 0.2942 - val_accuracy: 0.8425
Epoch 115/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3303 - accuracy: 0.8422 - val_loss: 0.2981 - val_accuracy: 0.8478
Epoch 116/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3322 - accuracy: 0.8418 - val_loss: 0.2882 - val_accuracy: 0.8475
Epoch 117/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3310 - accuracy: 0.8416 - val_loss: 0.3226 - val_accuracy: 0.8322
Epoch 118/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3300 - accuracy: 0.8418 - val_loss: 0.2628 - val_accuracy: 0.8831
Epoch 119/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3299 - accuracy: 0.8405 - val_loss: 0.3042 - val_ac

Epoch 169/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3266 - accuracy: 0.8407 - val_loss: 0.3344 - val_accuracy: 0.8325
Epoch 170/200
2371/2371 [==============================] - 6s 2ms/step - loss: 0.3252 - accuracy: 0.8413 - val_loss: 0.3370 - val_accuracy: 0.8329
Epoch 171/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3253 - accuracy: 0.8418 - val_loss: 0.2798 - val_accuracy: 0.8674
Epoch 172/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3258 - accuracy: 0.8419 - val_loss: 0.3666 - val_accuracy: 0.8322
Epoch 173/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3256 - accuracy: 0.8424 - val_loss: 0.3462 - val_accuracy: 0.8349
Epoch 174/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3262 - accuracy: 0.8417 - val_loss: 0.3148 - val_accuracy: 0.8348
Epoch 175/200
2371/2371 [==============================] - 6s 3ms/step - loss: 0.3258 - accuracy: 0.8405 - val_loss: 0

In [43]:
# Get model accuracy 
scores = model2.evaluate(test_features, test_labels)
print('\n')
print('accuracy=',scores[1])

# Get predictions from model
output = model2.predict_classes(test_features)

# Show confusion matrix
y_actu = pd.Series(test_labels, name='Actual')
y_pred = pd.Series(np.ndarray.flatten(output), name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)

num_positives =  (np.count_nonzero(y_actu))
num_negatives = y_actu.size - num_positives

TN = df_confusion[0][0]
TP = df_confusion[1][1]
FN = df_confusion[0][1]
FP = df_confusion[1][0]

precision = TP/(TP + FP)
recall = TP/(TP + FN)
specificity = TN/(TN + FP)
f1 = 2*((precision*recall)/(precision+recall))
print("Num Fraud: {}".format(num_positives))

print("Precision: ", precision)
print("Recall/Sensitivity: ", recall)
print("Specificity: ", specificity)
print("F1 Score: ", f1)

1158/1158 [==============================] - ETA: 0s - loss: 0.3207 - accuracy: 0.83 - 1s 1ms/step - loss: 0.3210 - accuracy: 0.8354


accuracy= 0.8354119658470154
Predicted      0     1
Actual                
0.0        28148  5921
1.0          176  2799
Num Fraud: 2975
Precision:  0.3209862385321101
Recall/Sensitivity:  0.9408403361344538
Specificity:  0.8262056414922657
F1 Score:  0.4786660966224882
